In [2]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [3]:
df = pd.read_csv('universities-clean.csv')

In [121]:
df.head()

,Country,University,Founded,Type,Enrollment,Link
0,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University


In [11]:
conn = psycopg2.connect(
    host="localhost",
    port=5435,
    user='erchey',
    password='Ercheypass123#',
    database="ercheydb"
)

In [12]:
cursor = conn.cursor()

In [124]:
create_query_file = open('./sql/postgrescreatetable.sql')
create_query = create_query_file.read()

In [125]:
create_query

'CREATE TABLE university (\n    id SERIAL PRIMARY KEY,\n    country VARCHAR(30) NOT NULL,\n    name VARCHAR(255) NOT NULL,\n    founded INT NOT NULL,\n    type VARCHAR(30) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    link VARCHAR(255) NOT NULL,\n    CONSTRAINT unique_combination UNIQUE (country, name)\n);'

In [126]:
cursor.execute(create_query)

In [127]:
conn.commit()

In [14]:
engine = create_engine("postgresql+psycopg2://", creator=lambda: conn)

In [15]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,country,name,founded,type,enrollment,link
0,1,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,2,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,3,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,4,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,5,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...,...
65,66,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/Open_University
66,67,United States,Texas A&M University,1876,Public,73284,https://en.wikipedia.org/wiki/Texas_A%26M_Univ...
67,68,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/University_of_th...
68,69,Venezuela,Universidad Central de Venezuela,1721,Public,41059,https://en.wikipedia.org/wiki/Universidad_Cent...


In [130]:
data = list(df.itertuples(name=None, index=None))

In [131]:
data

[('Albania',
  'University of Tirana',
  1957,
  'Public',
  35000,
  'https://en.wikipedia.org/wiki/University_of_Tirana'),
 ('Algeria',
  'Constantine University',
  1978,
  'Public',
  85000,
  'https://en.wikipedia.org/wiki/List_of_universities_in_Algeria'),
 ('Angola',
  'Agostinho Neto University',
  1962,
  'Public',
  29827,
  'https://en.wikipedia.org/wiki/Agostinho_Neto_University'),
 ('Argentina',
  'University of Buenos Aires',
  1821,
  'Public',
  311175,
  'https://en.wikipedia.org/wiki/University_of_Buenos_Aires'),
 ('Australia',
  'Monash University',
  1958,
  'Public',
  73807,
  'https://en.wikipedia.org/wiki/Monash_University'),
 ('Austria',
  'University of Vienna',
  1365,
  'Public',
  91000,
  'https://en.wikipedia.org/wiki/University_of_Vienna'),
 ('Bangladesh',
  'National University, Bangladesh',
  1992,
  'Public',
  2097182,
  'https://en.wikipedia.org/wiki/National_University,_Bangladesh'),
 ('Belarus',
  'Belarusian State University',
  1921,
  'Public',

In [137]:
merge_query_file = open("./sql/postgresmerge.sql")
merge_query = merge_query_file.read()
merge_query

"-- INSERT INTO university (country, name, founded, type, enrollment, link)\n-- VALUES (%s, %s, %s, %s, %s, %s)\n-- ON CONFLICT (country, name) DO UPDATE\n-- SET\n--     founded=EXCLUDED.founded,\n--     type=EXCLUDED.type,\n--     enrollment=EXCLUDED.enrollment,\n--     link=EXCLUDED.link;\n\nINSERT INTO university (country, name, founded, type, enrollment, link)\nVALUES ('Salsa', 'University of Salsa', 2002, 'Dance', 890, 'www.salsa.com');"

In [138]:
cursor.executemany(merge_query, data)

UniqueViolation: duplicate key value violates unique constraint "unique_combination"
DETAIL:  Key (country, name)=(Salsa, University of Salsa) already exists.


In [155]:
conn.commit()

In [9]:
uni_data = pd.read_sql("SELECT * FROM university;", con=engine)

In [10]:
uni_data.tail(20
              )

,id,country,name,founded,type,enrollment,link
50,51,Portugal,University of Lisbon,1288,Public,48100,https://en.wikipedia.org/wiki/University_of_Li...
51,52,Romania,Spiru Haret University,1991,Private,311928,https://en.wikipedia.org/wiki/Spiru_Haret_Univ...
52,53,Russia,Modern University for the Humanities,1992,Private,200000,https://en.wikipedia.org/wiki/Modern_Universit...
53,54,Saudi Arabia,King Abdulaziz University,1976,Public,177234,https://en.wikipedia.org/wiki/King_Abdulaziz_U...
54,55,Singapore,National University of Singapore,1905,Public,38300,https://en.wikipedia.org/wiki/National_Univers...
55,56,Slovenia,University of Ljubljana,1919,Public,48821,https://en.wikipedia.org/wiki/University_of_Lj...
56,57,South Africa,University of South Africa,1873,Public,328179,https://en.wikipedia.org/wiki/University_of_So...
57,58,South Korea,Korea National Open University,1972,Public,173758,https://en.wikipedia.org/wiki/Korea_National_O...
58,59,Spain,National University of Distance Education,1972,Public,260079,https://en.wikipedia.org/wiki/National_Univers...
59,60,Sweden,Lund University,1666,Public,30646,https://en.wikipedia.org/wiki/Lund_University
